In [ ]:
import numpy as np
import h5py
import scipy.io

In [ ]:
def load_dataset(idx, N):
    if idx==0:
        filepath = "/Users/cf/Dropbox/research/SonarLaptop/dataForPaper/twoLayer_train_N=80000.mat"
        f1 = h5py.File(filepath, mode='r')
    else:
        filepath = "/Users/cf/Dropbox/research/SonarLaptop/dataForPaper/Test Data/twoLayer_test_case"+ str(idx) +"_SNR_20.mat"
        f1 = h5py.File(filepath, mode='r')
       
    bs_2 = {}
   
    
 
    for k, v in f1.items():
        bs_2[k] = np.array(v)
    labels = bs_2['parameters'][6,:].transpose() if idx==0 else bs_2['parameters']
 
    ind = np.random.permutation(len(labels))
    Nd = np.amin([N,len(labels)])
    ind = ind[0:Nd]
   
    labels = labels[ind]
    labels = labels.reshape((Nd, 1))
    Y = labels.transpose()
    classes = np.unique(labels)
    for ck in range(len(classes)):
        Y = np.where(Y==classes[ck], ck, Y)

    y = np.array(Y[0], dtype=int)
    X = bs_2['signal'][:,ind].transpose()
   

    #making sure the labels are correct
    y=labels
    for i in range(N):
        if abs(labels[i]-1500)<20:
            y[i]=1500
        else:
            if abs(labels[i]-1575)<20:
                y[i]=1575
            else:
                if abs(labels[i]-1650)<20:
                    y[i]=1650
                else:
                    if abs(labels[i]-1800)<20:
                        y[i]=1800

    return X, y


In [ ]:
# Plot the training and test data
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm


X_train, y_train = load_dataset(0, 5)
X_test, y_test = load_dataset(4,5)
f, ax = plt.subplots(2, 2, sharey=False, sharex=False)
ax[0][0].plot(X_train[1:5,:].transpose(), color='orange')
ax[0][1].plot(y_train, color='yellow')
ax[1][0].plot(X_test[1:5,:].transpose(), color='green')
ax[1][1].plot(y_test, color='blue')

In [11]:
from time import time
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Benchmark metrics for each classifier
# Adapted from Peter Prettenhofer, et. al,
# https://scikit-learn.org/0.19/auto_examples/text/document_classification_20newsgroups.html
def benchmark(clf):
    score = [[] for x in range(5)]
    
    target_names=[]
    targets=np.unique(y_train);
    
    for i in range(len(targets)):
        target_names.append(np.str(targets[i]))
    print('_' * 80)
    print("Training: ")
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)
        
    t0 = time()
    print("Validation: ")
    pred=clf.predict(X_val)
    
    print(metrics.confusion_matrix(y_val, pred))
    print(metrics.classification_report(y_val, pred, target_names=target_names))
    score[0] = [metrics.accuracy_score(y_val, pred)]
    print("val score:   %0.3f" % score[0][0])
    
    print("Test1: ")
    t0 = time()
    pred = clf.predict(X_test1)
    test_time = time() - t0
    score[1] = [metrics.accuracy_score(y_test1, pred)]
    
    print("test time:  %0.3fs" % test_time)
    print("accuracy:   %0.3f" % score[1][0])
    #print("classification report:")
    #print(metrics.classification_report(y_test, pred, target_names=target_names))
    print("confusion matrix:")
    print(metrics.confusion_matrix(y_test1, pred))
    
    
    print("Test2: ")
    t0 = time()
    pred = clf.predict(X_test2)
    test_time = time() - t0
    score[2] = [metrics.accuracy_score(y_test2, pred)]
    
    print("test time:  %0.3fs" % test_time)
    print("accuracy:   %0.3f" % score[2][0])
    #print("classification report:")
    #print(metrics.classification_report(y_test, pred, target_names=target_names))
    print("confusion matrix:")
    print(metrics.confusion_matrix(y_test2, pred))
    print("Test3: ")
    t0 = time()
    pred = clf.predict(X_test3)
    test_time = time() - t0
    score[3] = [metrics.accuracy_score(y_test3, pred)]
    
    print("test time:  %0.3fs" % test_time)
    print("accuracy:   %0.3f" % score[3][0])
    #print("classification report:")
    #print(metrics.classification_report(y_test3, pred, target_names=target_names))
    print("confusion matrix:")
    print(metrics.confusion_matrix(y_test3, pred))
    
    print("Test4: ")
    t0 = time()
    pred = clf.predict(X_test4)
    test_time = time() - t0
    score[4] = [metrics.accuracy_score(y_test4, pred)]
    
    print("test time:  %0.3fs" % test_time)
    print("accuracy:   %0.3f" % score[4][0])
    #print("classification report:")
    #print(metrics.classification_report(y_test, pred, target_names=target_names))
    print("confusion matrix:")
    print(metrics.confusion_matrix(y_test4, pred))
  
    
    clf_descr = str(str(clf.estimator))
    clf_rep = str(metrics.classification_report(y_test4, pred, target_names=target_names))
    clf_cm = str(metrics.confusion_matrix(y_test4, pred))
    return clf_descr, score, train_time, test_time, clf_rep, clf_cm
      
      

In [5]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform, expon
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis


# list of (estimator, param_dist), where param_dist is used  in RandomizedSearchCV
classifiers = [
#      (GaussianProcessClassifier(warm_start=True, n_jobs=-1, random_state=42),{
#         'multi_class': ["one_vs_rest", "one_vs_one"],
#         'n_restarts_optimizer':[0, 1]
#      }),
     (KNeighborsClassifier(), {
         'n_neighbors': sp_randint(8, 10),
         'weights': ['uniform', 'distance'],
         'algorithm':['ball_tree', 'kd_tree'],
     }),
    (NearestCentroid(), {
         'metric': ['euclidean', 'manhattan','minkowski','chebyshev'],
         'shrink_threshold': [None, .001, .0001, .1, .01]
     }),
    (SVC(kernel='linear'), {
        'C': np.logspace(-1, 3, 100),
        'tol': np.logspace(-8, -2, 100)
    }),
    (SVC(random_state=42, gamma='scale'), {
        'C': [.1, 1, 10, 100, 1000],
        'kernel': ['rbf', 'poly', 'linear', 'sigmoid'], 
        'class_weight':['balanced', None],
        'tol': np.logspace(-8, -2, 100)
    }),
    (MLPClassifier(max_iter=100000), {
        'hidden_layer_sizes': [(200), (50), (100,)],
        'alpha': expon(scale=.1),
        'learning_rate': ['constant','adaptive'],
    }),
    (DecisionTreeClassifier(),{
        'criterion': ['gini', 'entropy'],
        'splitter': ['best','random'],
        'max_features':[None, 'auto', 'sqrt', 'log2'],
    }),
    (RandomForestClassifier(max_depth=5, n_jobs=-1), {
        "max_depth": sp_randint(2, 20),
        "n_estimators": sp_randint(2, 50),
        "max_features": ['auto', 'log2'],
        'class_weight': ['balanced', 'balanced_subsample'],
        'criterion': ['gini', 'entropy']
    }),
     (GaussianNB(), {
         'var_smoothing': np.logspace(-12, -8, 100)
     }), 
     (LinearDiscriminantAnalysis(n_components=2), {
         'solver': ['svd', 'lsqr'],
         'tol': np.logspace(-10, -2, 100)
     }),
     (LogisticRegression(solver='newton-cg', random_state=0, max_iter=100000), {
        'C': [.1, 1, 10, 100, 1000],
         'multi_class': ["auto", "ovr","multinomial"],
         'solver': ['sag', 'saga', 'newton-cg'],
         'tol': np.logspace(-6, -4, 100)
     }),
]
names = [e.__class__.__name__ for e, g in classifiers]

In [9]:
from sklearn.model_selection import train_test_split
# Load training and test data
X,y=load_dataset(0, 5000)

#Split into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=42)
X_test1, y_test1= load_dataset(1, 20000)
X_test2, y_test2= load_dataset(2, 20000)
X_test3, y_test3= load_dataset(3, 20000)
X_test4, y_test4= load_dataset(4, 20000)

In [ ]:


print(np.unique(y_train))
print(np.unique(y_val))
print(np.unique(y_test1))
print(y_test1.shape)

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import RandomizedSearchCV
d_bs_2= X_train,y_train
datasets = [d_bs_2]

data_name=['BS 2 layer']
dsets=datasets
dset_name=data_name

grid_searches = [dict() for x in range(len(dsets))]

results = [[] for x in range(len(dsets))]



# iterate over datasets
for ds_cnt, (X, y) in enumerate(dsets):
    
    print('\n Dataset:', data_name[ds_cnt],'\n')
 
    # preprocess dataset
    X = StandardScaler().fit_transform(X)
    
    # iterate over classifiers
    for est_idx, (name, (estimator, param_grid)) in \
            enumerate(zip(names, classifiers)):
        print(name)
    
        # Perform randomized grid search over possible hyperparameters
        clf = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, n_iter=20, cv=5,
                           iid='true', verbose=0, n_jobs=-1)
        
       
        
        with ignore_warnings(category=ConvergenceWarning):
            results[ds_cnt].append(benchmark(clf))
        
        # Make predictions for the four test cases
        #score = [clf.score(X_val, y_val), clf.score(X_test1, y_test1), clf.score(X_test2, y_test2), clf.score(X_test3, y_test3),clf.score(X_test4, y_test4)]  
        
        #print(score)
        
        # Store the classifier
        grid_searches[ds_cnt][name] = clf
       
    


 Dataset: BS 2 layer 

KNeighborsClassifier
________________________________________________________________________________
Training: 


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


train time: 34.124s
Validation: 
[[240   1   0   0]
 [101 135  14   0]
 [ 74 117  67  10]
 [ 77  10  39 115]]
              precision    recall  f1-score   support

      1500.0       0.49      1.00      0.65       241
      1575.0       0.51      0.54      0.53       250
      1650.0       0.56      0.25      0.35       268
      1800.0       0.92      0.48      0.63       241

    accuracy                           0.56      1000
   macro avg       0.62      0.57      0.54      1000
weighted avg       0.62      0.56      0.53      1000

val score:   0.557
Test1: 
test time:  129.471s
accuracy:   0.594
confusion matrix:
[[4810   58    0    0]
 [1733 2771  318   18]
 [1869 1556 1306  329]
 [1301  188  747 2996]]
Test2: 
test time:  129.207s
accuracy:   0.571
confusion matrix:
[[4741   79    0    0]
 [1795 2822  344   31]
 [2076 1744 1122  234]
 [1431  205  646 2730]]
Test3: 
test time:  126.272s
accuracy:   0.463
confusion matrix:
[[4396  261   49  110]
 [2134 2335  330  189]
 [2146 16

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train time: 2.745s
Validation: 
[[184   4  49   4]
 [ 70 106  72   2]
 [  6  66 160  36]
 [  0   0  58 183]]
              precision    recall  f1-score   support

      1500.0       0.71      0.76      0.73       241
      1575.0       0.60      0.42      0.50       250
      1650.0       0.47      0.60      0.53       268
      1800.0       0.81      0.76      0.79       241

    accuracy                           0.63      1000
   macro avg       0.65      0.64      0.64      1000
weighted avg       0.64      0.63      0.63      1000

val score:   0.633
Test1: 
test time:  0.104s
accuracy:   0.666
confusion matrix:
[[3608  102  921  237]
 [1057 2152 1471  160]
 [  69  822 3202  967]
 [   0   14  854 4364]]
Test2: 
test time:  0.103s
accuracy:   0.645
confusion matrix:
[[3402   90 1126  202]
 [1131 2157 1566  138]
 [  93  935 3229  919]
 [   0    8  884 4120]]
Test3: 
test time:  0.102s
accuracy:   0.534
confusion matrix:
[[2306  119 1057 1334]
 [ 825 2048  788 1327]
 [  13  899 3436

...

train time: 1.628s
Validation: 
[[187   4  50   0]
 [ 93  94  61   2]
 [ 12  74 154  28]
 [  0   1  58 182]]
              precision    recall  f1-score   support

      1500.0       0.64      0.78      0.70       241
      1575.0       0.54      0.38      0.44       250
      1650.0       0.48      0.57      0.52       268
      1800.0       0.86      0.76      0.80       241

    accuracy                           0.62      1000
   macro avg       0.63      0.62      0.62      1000
weighted avg       0.62      0.62      0.61      1000

val score:   0.617
Test1: 
test time:  0.852s
accuracy:   0.658
confusion matrix:
[[3687   55 1122    4]
 [1497 2076 1156  111]
 [  83 1008 3105  864]
 [   0   21  915 4296]]
Test2: 
test time:  0.847s
accuracy:   0.633
confusion matrix:
[[3461   60 1295    4]
 [1503 2083 1324   82]
 [ 105 1166 3094  811]
 [   0   20  968 4024]]
Test3: 
test time:  0.845s
accuracy:   0.513
confusion matrix:
[[2339  823  746  908]
 [1132 1890  980  986]
 [  49 1093 3311

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


train time: 18.489s
Validation: 
[[219  21   1   0]
 [ 70 132  40   8]
 [ 14  62 140  52]
 [  2  10  56 173]]
              precision    recall  f1-score   support

      1500.0       0.72      0.91      0.80       241
      1575.0       0.59      0.53      0.56       250
      1650.0       0.59      0.52      0.55       268
      1800.0       0.74      0.72      0.73       241

    accuracy                           0.66      1000
   macro avg       0.66      0.67      0.66      1000
weighted avg       0.66      0.66      0.66      1000

val score:   0.664
Test1: 
test time:  0.027s
accuracy:   0.264
confusion matrix:
[[1657  772  956 1483]
 [1585  717 1021 1517]
 [1608  705 1056 1691]
 [1624  728 1033 1847]]
Test2: 
test time:  0.023s
accuracy:   0.256
confusion matrix:
[[1622  708  997 1493]
 [1610  722 1016 1644]
 [1642  724 1077 1733]
 [1512  741 1053 1706]]
Test3: 
test time:  0.022s
accuracy:   0.259
confusion matrix:
[[1646  733  980 1457]
 [1601  700 1004 1683]
 [1590  750 112

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train time: 2752.060s
Validation: 
[[208  25   3   5]
 [ 76 119  52   3]
 [ 13  72 150  33]
 [  0   5  36 200]]
              precision    recall  f1-score   support

      1500.0       0.70      0.86      0.77       241
      1575.0       0.54      0.48      0.51       250
      1650.0       0.62      0.56      0.59       268
      1800.0       0.83      0.83      0.83       241

    accuracy                           0.68      1000
   macro avg       0.67      0.68      0.67      1000
weighted avg       0.67      0.68      0.67      1000

val score:   0.677
Test1: 
test time:  0.018s
accuracy:   0.574
confusion matrix:
[[3313 1051  329  175]
 [1582 1811 1259  188]
 [ 628 1077 2406  949]
 [  79  201 1006 3946]]
Test2: 
test time:  0.019s
accuracy:   0.557
confusion matrix:
[[3243 1079  327  171]
 [1667 1753 1382  190]
 [ 679 1161 2448  888]
 [  80  191 1038 3703]]
Test3: 
test time:  0.019s
accuracy:   0.415
confusion matrix:
[[2172  998  527 1119]
 [1364 1456 1080 1088]
 [ 851 1312 2

In [ ]:
from sklearn.metrics import confusion_matrix
# iterate over datasets
for ds_cnt, (X, y) in enumerate(dsets):
    for est_idx, (name, (estimator, param_grid)) in \
            enumerate(zip(names, classifiers)):
        clf=grid_searches[ds_cnt][name]
        clf.fit(X_train, y_train)
    
        score = [clf.score(X_test, y_test), clf.score(X_test1, y_test1), clf.score(X_test2, y_test2), clf.score(X_test3, y_test3),clf.score(X_test4, y_test4)]  
        print(name)
        print(score)
    
        conf = confusion_matrix(y_test1, clf.predict(X_test1))
        print(conf)

In [ ]:
import datetime
print(n)
print("\\begin{filecontents}{"+names[nc]+"-bs.dat}\n noise 	snr accuracy") 
for nl in range(len(noiselevel)):
    clf_descr, score, train_time, test_time, clf_rep, clf_cm=results[0][nc]

    print(str(noiselevel[nl])+" "+ str(18-nl)+" "+ str(score*100))
print("\end{filecontents}")

In [35]:
import datetime
import time
for nc in range(10):
    clf_descr, score, train_time, test_time, clf_rep, clf_cm=results[0][nc]

    print(names[nc]+"&"+str(datetime.timedelta(seconds=np.round(train_time)))+"&"+str(np.round(score[0][0]*100,decimals=2))+"&"
                           + str(np.round(score[1][0]*100,decimals=2))+"&"
                           + str(np.round(score[2][0]*100,decimals=2))+"&"
                           + str(np.round(score[3][0]*100,decimals=2))+"&"
                           + str(np.round(score[4][0]*100,decimals=2))+"\\"+"\\")
          


KNeighborsClassifier&0:00:34&55.7&59.41&57.08&46.29&57.36\\
NearestCentroid&0:00:03&63.3&66.63&64.54&53.37&64.67\\
SVC&0:02:18&67.4&59.44&58.14&42.98&58.36\\
SVC&0:02:05&77.3&76.62&75.76&56.63&76.19\\
MLPClassifier&0:05:38&73.6&73.04&72.43&51.47&73.43\\
DecisionTreeClassifier&0:00:10&55.1&55.95&54.36&43.17&53.94\\
RandomForestClassifier&0:00:07&71.5&71.74&70.15&53.86&71.44\\
GaussianNB&0:00:02&61.7&65.82&63.31&51.34&63.08\\
LinearDiscriminantAnalysis&0:00:18&66.4&26.38&25.64&25.93&26.78\\
LogisticRegression&0:45:52&67.7&57.38&55.74&41.54&56.36\\


In [ ]:
import pandas as pd
def score_summary(grid_searches, sort_by='mean_test_score'):
        frames = []
        for name, grid_search in grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
        df = pd.concat(frames)
        
        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_score', 'index'], 1)
        
        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df

In [ ]:
df=score_summary(grid_searches[0])
df.params[0]